In [11]:
# Este código realiza una API a finviz para listar empresas biotechnológicas
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import matplotlib as plt
import warnings
import seaborn as sns
import requests
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import pymysql
import seaborn as sns
from datetime import datetime
import re
import time
import ast
from datetime import datetime
from finvizfinance.screener.overview import Overview
from finvizfinance.quote import finvizfinance
import yfinance as yf
from yahoo_fin import news as yf_news
now = datetime.now()
# Format the date and time as a string
date_str = now.strftime("%Y%m%d")
pd.set_option("display.max_colwidth", 150)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
from finvizfinance.screener.overview import Overview

# Initialize the overview screener
foverview = Overview()

# Set general filters for biotechnology companies
filters_dict = {
    'Sector': 'Healthcare',
    'Industry': 'Biotechnology'
}
# Apply the filters
foverview.set_filter(filters_dict=filters_dict)

# Retrieve the screener data as a DataFrame
df = foverview.screener_view()
print('\n')
print('Encontradas: '+ str(len(df)) + ' Empreas Biotech ')
df = df.drop(columns=["Sector","Market Cap", "P/E", "Price", "Change", "Volume"])

[Info] loading page [##############################] 34/34 

Encontradas: 669 Empreas Biotech 


In [3]:
def business_summary(ticker):
    try:
        stock = yf.Ticker(ticker)
        # Validar si `info` tiene datos
        if not stock.info:
            print(f"Información no disponible para el ticker: {ticker}")
            return {"longBusinessSummary": None, "fullTimeEmployees": None}
        # Extraer datos necesarios
        summary = stock.info.get("longBusinessSummary", "None")
        employees = stock.info.get("fullTimeEmployees", "None")
        web = stock.info.get("website","NaN")
        volume = stock.info.get("averageVolume","None") 
        marketcap = stock.info.get("marketCap", "None")
        ebitda = stock.info.get("ebitda","None")
        revenue = stock.info.get("totalRevenue","None")
        ipo = stock.info.get("firstTradeDateEpochUtc","None")
        return {"BusinessSummary": summary, "Employees": employees, "web": web, "volume":volume, "marketcap": marketcap, "ebitda": ebitda, "revenue": revenue, "ipo":ipo}
    except Exception as e:
        print(f"Error al obtener datos para el ticker {ticker}: {e}")
        return {"BusinessSummary": None, "Employees": None}

In [4]:
results = []
for ticker in df["Ticker"]:
    result = business_summary(ticker)
    results.append(result)
    time.sleep(2)  # Pausa de 2 segundos entre solicitudes

df_summary = pd.DataFrame(results)
df = pd.concat([df, df_summary], axis=1)

In [5]:
print(df.dtypes)

Ticker             object
Company            object
Industry           object
Country            object
BusinessSummary    object
Employees          object
web                object
volume              int64
marketcap          object
ebitda             object
revenue            object
ipo                 int64
dtype: object


In [6]:
#formateo de data
numeric_columns = ["Employees", "ebitda", "revenue"]
for col in numeric_columns:
    if col not in df.columns:
        df[col] = None  # Crear la columna si no existe para evitar errores

# Convertir columnas a numéricas (cambiando errores a NaN si el valor no es convertible)
df["Employees"] = pd.to_numeric(df["Employees"], errors="coerce").fillna(0).astype(int)
df["volume"] = pd.to_numeric(df["volume"], errors="coerce").fillna(0).astype(int)

df["ebitda"] = pd.to_numeric(df["ebitda"], errors="coerce").fillna(0)
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce").fillna(0)
df["marketcap"] = pd.to_numeric(df["marketcap"], errors="coerce").fillna(0)
# Renombrar "Market Cap" a "MarketCapM" y dividir por 1 millón
df.rename(columns={"marketcap": "MarketCap_M"}, inplace=True)
df["MarketCap_M"] = round(df["MarketCap_M"] / 1_000_000,2)

df["ebitdaM"] = round(df["ebitda"] / 1_000_000,2)
df["Volume_m"] = round(df["volume"] / 1_000,0)
df["revenueM"] = round(df["revenue"] / 1_000_000,2)
# Convertir columna "ip" de Unix Epoch a formato de fecha legible
if "ipo" in df.columns:
    df["ipo_date"] = pd.to_datetime(df["ipo"], unit="s", errors="coerce").dt.date
    df["ipo_date"] = pd.to_datetime(df["ipo_date"])
df = df.drop(columns=["revenue", "ipo","ebitda","volume"])

In [7]:
desired_order = [
    "Ticker", "Company",  "ipo_date", "MarketCap_M", "Employees", "Volume_m",  "revenueM", "ebitdaM", "web", "Country", "Industry", "BusinessSummary"
]
df = df[desired_order]

In [8]:
df

,Ticker,Company,ipo_date,MarketCap_M,Employees,Volume_m,revenueM,ebitdaM,web,Country,Industry,BusinessSummary
0,AADI,Aadi Bioscience Inc,2018-02-16,58.66,53,117.00,25.07,-63.40,https://aadibio.com,USA,Biotechnology,"Aadi Bioscience, Inc., a biopharmaceutical company, engages in developing and commercializing precision therapies for genetically defined cancers ..."
1,ABCL,AbCellera Biologics Inc,2020-12-11,844.75,586,2208.00,32.96,-222.24,https://www.abcellera.com,Canada,Biotechnology,AbCellera Biologics Inc. builds an engine for antibody drug discovery and development. Its engine discovers antibodies from natural immune respons...
2,ABEO,Abeona Therapeutics Inc,1980-09-19,256.04,84,276.00,0.00,-54.75,https://www.abeonatherapeutics.com,USA,Biotechnology,"Abeona Therapeutics Inc., a clinical-stage biopharmaceutical company, develops gene and cell therapies for life-threatening diseases. Its lead cli..."
3,ABOS,Acumen Pharmaceuticals Inc,2021-07-01,118.36,51,219.00,0.00,-92.84,https://acumenpharm.com,USA,Biotechnology,"Acumen Pharmaceuticals, Inc., a clinical-stage biopharmaceutical company, develops targeted therapies for the treatment of Alzheimer's disease. Th..."
4,ABP,Abpro Holdings Inc,2022-03-09,65.94,15,28.00,0.07,-11.31,https://abpro.com,USA,Biotechnology,"Abpro Holdings, Inc., a biotechnology company, engages in developing antibody therapeutics for patients with immuno-oncology, ophthalmology, and a..."
5,ABSI,Absci Corp,2021-07-22,439.90,155,1380.00,4.21,-89.71,https://www.absci.com,USA,Biotechnology,Absci Corporation operates as a data-first generative artificial intelligence (AI) drug creation company in the United States. The company combine...
6,ABUS,Arbutus Biopharma Corp,2007-07-26,672.70,73,974.00,6.74,-77.14,https://www.arbutusbio.com,USA,Biotechnology,"Arbutus Biopharma Corporation, a biopharmaceutical company, develops novel therapeutics for chronic Hepatitis B virus (HBV) infection in the Unite..."
7,ABVC,ABVC BioPharma Inc,2004-11-10,6.24,16,248.00,0.14,-8.51,https://www.abvcpharma.com,USA,Biotechnology,"ABVC BioPharma, Inc., a clinical stage biopharmaceutical company, develops drugs and medical devices to fulfill unmet medical needs in the United ..."
8,ABVX,Abivax ADR,2023-10-20,500.36,62,131.00,9.03,-169.87,https://www.abivax.com,France,Biotechnology,"ABIVAX Société Anonyme, a clinical-stage biotechnology company, focuses on developing therapeutics that harness the body's natural regulatory mech..."
9,ACAD,Acadia Pharmaceuticals Inc,2004-05-27,2885.24,620,1469.00,929.24,124.54,https://www.acadia.com,USA,Biotechnology,"ACADIA Pharmaceuticals Inc., a biopharmaceutical company, focuses on the development and commercialization innovative medicines that address unmet..."


In [9]:
#Save to directory
output_directory = "output"
date_string = datetime.now().strftime("%Y-%m-%d")
file_name = f"df_allBiotech_{date_string}_.csv"


# Construct the full path
file_path = os.path.join(output_directory, file_name)

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame has been saved to: {file_path}")

DataFrame has been saved to: output\df_allBiotech_2024-12-17_.csv


In [10]:
print(df.dtypes)

Ticker                     object
Company                    object
ipo_date           datetime64[ns]
MarketCap_M               float64
Employees                   int32
Volume_m                  float64
revenueM                  float64
ebitdaM                   float64
web                        object
Country                    object
Industry                   object
BusinessSummary            object
dtype: object
